# DeepAR - Experiments

### DeepAR (Salinas et al., 2017)

The **DeepAR** algorithm uses autoregressive features that are used as the inputs to a recurrent neural network. The RNN uses LSTM cells as default but can also changed to GRU cells. In both cases previous time points are taken as input. 

- [`DeepAREestimator`](https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html)

In [1]:
# standard imports
import numpy as np
import pandas as pd

# json
import json

# gluon data
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# model imports
# from gluonts.model.deepstate import DeepStateEstimator
# from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint


### define main function ### 
def main(data="m4_weekly", seed=42, epochs=100, batches=50):
    
    dataset = get_dataset(data, regenerate=False)
    mx.random.seed(seed)
    np.random.seed(seed)
    
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=epochs,      
        num_batches_per_epoch=batches,      
        learning_rate=1e-3,
    )
    
    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True   
    )
        
    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    output["epochs"] = epochs
    return(output)


INFO:root:Using CPU


In [150]:
# seed 42
results42 = []

for i in range(100, 401, 50):
    print("Epochs:", i)
    res = main(data="m4_daily", seed=42, epochs=i, batches=50)
    pprint(res)
    results42.append(res)
    
results42

Epochs: 100


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 12.48it/s, avg_epoch_loss=8]
INFO:root:Epoch[0] Elapsed time 4.012 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.998756
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 12.15it/s, avg_epoch_loss=6.52]
INFO:root:Epoch[1] Elapsed time 4.130 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.519081
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 11.03it/s, avg_epoch_loss=6.53]
INFO:root:Epoch[2] Elapsed time 4.536 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.527947
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 11.59it/s, avg_epoch_loss=6.34]
INFO:root:Epoch[3] Elapsed time 4.319 seconds
INFO:ro

INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=5.687431
INFO:root:Epoch[74] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 11.39it/s, avg_epoch_loss=5.55]
INFO:root:Epoch[74] Elapsed time 4.394 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=5.545422
INFO:root:Epoch[75] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 10.13it/s, avg_epoch_loss=5.71]
INFO:root:Epoch[75] Elapsed time 4.943 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=5.707698
INFO:root:Epoch[76] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 10.60it/s, avg_epoch_loss=5.66]
INFO:root:Epoch[76] Elapsed time 4.725 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=5.663023
INFO:root:Epoch[77] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 11.00it/s, avg_epoch_loss=5.54]
INFO:root:Epoch[77] Elapsed time 4.551 seconds
INFO:root:Epoch[77] Evaluation metric 'epoch_loss'=5.535846
INFO:root:Epoch[78] Learning rate is 0.001
100%|██████████| 50/5

{'MASE': 4.90190946, 'sMAPE': 0.04442959, 'MSIS': 69.7043414, 'wQuantileLoss[0.5]': 0.04011688, 'wQuantileLoss[0.9]': 0.02239325, 'epochs': 100}
Epochs: 150


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 11.74it/s, avg_epoch_loss=8]
INFO:root:Epoch[0] Elapsed time 4.276 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.998756
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 11.06it/s, avg_epoch_loss=6.52]
INFO:root:Epoch[1] Elapsed time 4.529 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.519081
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:05<00:00,  9.91it/s, avg_epoch_loss=6.53]
INFO:root:Epoch[2] Elapsed time 5.050 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.527947
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 10.37it/s, avg_epoch_loss=6.34]
INFO:root:Epoch[3] Elapsed time 4.827 seconds
INFO:ro

...

INFO:root:Epoch[219] Elapsed time 5.011 seconds
INFO:root:Epoch[219] Evaluation metric 'epoch_loss'=5.458229
INFO:root:Epoch[220] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 10.03it/s, avg_epoch_loss=5.39]
INFO:root:Epoch[220] Elapsed time 5.000 seconds
INFO:root:Epoch[220] Evaluation metric 'epoch_loss'=5.386741
INFO:root:Epoch[221] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 10.44it/s, avg_epoch_loss=5.63]
INFO:root:Epoch[221] Elapsed time 4.793 seconds
INFO:root:Epoch[221] Evaluation metric 'epoch_loss'=5.634480
INFO:root:Epoch[222] Learning rate is 5e-05
100%|██████████| 50/50 [00:05<00:00,  9.76it/s, avg_epoch_loss=5.43]
INFO:root:Epoch[222] Elapsed time 5.134 seconds
INFO:root:Epoch[222] Evaluation metric 'epoch_loss'=5.433138
INFO:root:Epoch[223] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.29it/s, avg_epoch_loss=5.49]
INFO:root:Epoch[223] Elapsed time 9.451 seconds
INFO:root:Epoch[223] Evaluation metric 'epoch_loss'=5.492531
INF

INFO:root:Epoch[293] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 10.24it/s, avg_epoch_loss=5.52]
INFO:root:Epoch[293] Elapsed time 4.888 seconds
INFO:root:Epoch[293] Evaluation metric 'epoch_loss'=5.517908
INFO:root:Epoch[294] Learning rate is 5e-05
100%|██████████| 50/50 [00:05<00:00,  9.59it/s, avg_epoch_loss=5.4]
INFO:root:Epoch[294] Elapsed time 5.221 seconds
INFO:root:Epoch[294] Evaluation metric 'epoch_loss'=5.398970
INFO:root:Epoch[295] Learning rate is 5e-05
100%|██████████| 50/50 [00:06<00:00,  7.68it/s, avg_epoch_loss=5.51]
INFO:root:Epoch[295] Elapsed time 6.523 seconds
INFO:root:Epoch[295] Evaluation metric 'epoch_loss'=5.514793
INFO:root:Epoch[296] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.49it/s, avg_epoch_loss=5.54]
INFO:root:Epoch[296] Elapsed time 9.112 seconds
INFO:root:Epoch[296] Evaluation metric 'epoch_loss'=5.535820
INFO:root:Epoch[297] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 10.25it/s, avg_epoch_loss=5.31]


INFO:root:Epoch[366] Elapsed time 5.306 seconds
INFO:root:Epoch[366] Evaluation metric 'epoch_loss'=5.494510
INFO:root:Epoch[367] Learning rate is 5e-05
100%|██████████| 50/50 [00:07<00:00,  6.72it/s, avg_epoch_loss=5.52]
INFO:root:Epoch[367] Elapsed time 7.446 seconds
INFO:root:Epoch[367] Evaluation metric 'epoch_loss'=5.519973
INFO:root:Epoch[368] Learning rate is 5e-05
100%|██████████| 50/50 [00:05<00:00,  8.67it/s, avg_epoch_loss=5.35]
INFO:root:Epoch[368] Elapsed time 5.780 seconds
INFO:root:Epoch[368] Evaluation metric 'epoch_loss'=5.351171
INFO:root:Epoch[369] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 10.00it/s, avg_epoch_loss=5.5]
INFO:root:Epoch[369] Elapsed time 5.006 seconds
INFO:root:Epoch[369] Evaluation metric 'epoch_loss'=5.500445
INFO:root:Epoch[370] Learning rate is 5e-05
100%|██████████| 50/50 [00:05<00:00,  9.84it/s, avg_epoch_loss=5.48]
INFO:root:Epoch[370] Elapsed time 5.088 seconds
INFO:root:Epoch[370] Evaluation metric 'epoch_loss'=5.480932
INFO

{'MASE': 3.49450778, 'sMAPE': 0.03328832, 'MSIS': 43.73285373, 'wQuantileLoss[0.5]': 0.03004035, 'wQuantileLoss[0.9]': 0.01436039, 'epochs': 400}


[{'MASE': 4.90190946,
  'sMAPE': 0.04442959,
  'MSIS': 69.7043414,
  'wQuantileLoss[0.5]': 0.04011688,
  'wQuantileLoss[0.9]': 0.02239325,
  'epochs': 100},
 {'MASE': 3.56823832,
  'sMAPE': 0.03372614,
  'MSIS': 46.58285017,
  'wQuantileLoss[0.5]': 0.03048994,
  'wQuantileLoss[0.9]': 0.01511912,
  'epochs': 150},
 {'MASE': 3.56498368,
  'sMAPE': 0.03368993,
  'MSIS': 46.42841039,
  'wQuantileLoss[0.5]': 0.03045386,
  'wQuantileLoss[0.9]': 0.01508755,
  'epochs': 200},
 {'MASE': 3.56713179,
  'sMAPE': 0.03371625,
  'MSIS': 46.45630644,
  'wQuantileLoss[0.5]': 0.03047929,
  'wQuantileLoss[0.9]': 0.01507606,
  'epochs': 250},
 {'MASE': 3.50397495,
  'sMAPE': 0.03335477,
  'MSIS': 43.92927541,
  'wQuantileLoss[0.5]': 0.03012155,
  'wQuantileLoss[0.9]': 0.01448445,
  'epochs': 300},
 {'MASE': 3.50300612,
  'sMAPE': 0.03335573,
  'MSIS': 43.93653794,
  'wQuantileLoss[0.5]': 0.03013073,
  'wQuantileLoss[0.9]': 0.0145014,
  'epochs': 350},
 {'MASE': 3.49450778,
  'sMAPE': 0.03328832,
  'MSIS':

In [3]:
# seed 43
results43 = []

for i in range(100, 401, 50):
    print("Epochs:", i)
    res = main(data="m4_daily", seed=43, epochs=i, batches=50)
    pprint(res)
    results43.append(res)
    
results43

Epochs: 100


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:05<00:00,  9.00it/s, avg_epoch_loss=7.35]
INFO:root:Epoch[0] Elapsed time 5.559 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.349555
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 10.65it/s, avg_epoch_loss=6.54]
INFO:root:Epoch[1] Elapsed time 4.704 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.537140
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:05<00:00,  9.02it/s, avg_epoch_loss=6.7]
INFO:root:Epoch[2] Elapsed time 5.545 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.702750
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 10.93it/s, avg_epoch_loss=6.39]
INFO:root:Epoch[3] Elapsed time 4.582 seconds
INFO:

INFO:root:Epoch[73] Elapsed time 3.855 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=5.711529
INFO:root:Epoch[74] Learning rate is 0.0005
100%|██████████| 50/50 [00:04<00:00, 11.60it/s, avg_epoch_loss=5.5]
INFO:root:Epoch[74] Elapsed time 4.318 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=5.501224
INFO:root:Epoch[75] Learning rate is 0.0005
100%|██████████| 50/50 [00:04<00:00, 12.02it/s, avg_epoch_loss=5.73]
INFO:root:Epoch[75] Elapsed time 4.163 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=5.729530
INFO:root:Epoch[76] Learning rate is 0.0005
100%|██████████| 50/50 [00:04<00:00, 12.27it/s, avg_epoch_loss=5.63]
INFO:root:Epoch[76] Elapsed time 4.082 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=5.627868
INFO:root:Epoch[77] Learning rate is 0.0005
100%|██████████| 50/50 [00:04<00:00, 11.80it/s, avg_epoch_loss=5.56]
INFO:root:Epoch[77] Elapsed time 4.245 seconds
INFO:root:Epoch[77] Evaluation metric 'epoch_loss'=5.555615
INFO:root:Epoc

{'MASE': 4.00380739,
 'MSIS': 53.09362333,
 'epochs': 100,
 'sMAPE': 0.03668879,
 'wQuantileLoss[0.5]': 0.0307505,
 'wQuantileLoss[0.9]': 0.01408046}
Epochs: 150


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 12.80it/s, avg_epoch_loss=7.35]
INFO:root:Epoch[0] Elapsed time 3.954 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.349555
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 11.90it/s, avg_epoch_loss=6.54]
INFO:root:Epoch[1] Elapsed time 4.209 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.537140
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:06<00:00,  7.17it/s, avg_epoch_loss=6.7]
INFO:root:Epoch[2] Elapsed time 6.982 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.702750
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 11.41it/s, avg_epoch_loss=6.39]
INFO:root:Epoch[3] Elapsed time 4.389 seconds
INFO:

...

INFO:root:Epoch[219] Evaluation metric 'epoch_loss'=5.415066
INFO:root:Epoch[220] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 11.78it/s, avg_epoch_loss=5.57]
INFO:root:Epoch[220] Elapsed time 4.251 seconds
INFO:root:Epoch[220] Evaluation metric 'epoch_loss'=5.566513
INFO:root:Epoch[221] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 11.65it/s, avg_epoch_loss=5.56]
INFO:root:Epoch[221] Elapsed time 4.299 seconds
INFO:root:Epoch[221] Evaluation metric 'epoch_loss'=5.560207
INFO:root:Epoch[222] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 10.54it/s, avg_epoch_loss=5.39]
INFO:root:Epoch[222] Elapsed time 4.749 seconds
INFO:root:Epoch[222] Evaluation metric 'epoch_loss'=5.394659
INFO:root:Epoch[223] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 11.61it/s, avg_epoch_loss=5.6]
INFO:root:Epoch[223] Elapsed time 4.317 seconds
INFO:root:Epoch[223] Evaluation metric 'epoch_loss'=5.598100
INFO:root:Epoch[224] Learning rate is 5e-05
100%|███

100%|██████████| 50/50 [00:04<00:00, 11.37it/s, avg_epoch_loss=5.46]
INFO:root:Epoch[293] Elapsed time 4.402 seconds
INFO:root:Epoch[293] Evaluation metric 'epoch_loss'=5.458392
INFO:root:Epoch[294] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 11.60it/s, avg_epoch_loss=5.45]
INFO:root:Epoch[294] Elapsed time 4.316 seconds
INFO:root:Epoch[294] Evaluation metric 'epoch_loss'=5.453703
INFO:root:Epoch[295] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 11.31it/s, avg_epoch_loss=5.56]
INFO:root:Epoch[295] Elapsed time 4.427 seconds
INFO:root:Epoch[295] Evaluation metric 'epoch_loss'=5.556143
INFO:root:Epoch[296] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 12.10it/s, avg_epoch_loss=5.4]
INFO:root:Epoch[296] Elapsed time 4.138 seconds
INFO:root:Epoch[296] Evaluation metric 'epoch_loss'=5.404588
INFO:root:Epoch[297] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 11.87it/s, avg_epoch_loss=5.53]
INFO:root:Epoch[297] Elapsed time 4.219 seco

INFO:root:Epoch[366] Evaluation metric 'epoch_loss'=5.538510
INFO:root:Epoch[367] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 11.43it/s, avg_epoch_loss=5.48]
INFO:root:Epoch[367] Elapsed time 4.391 seconds
INFO:root:Epoch[367] Evaluation metric 'epoch_loss'=5.475311
INFO:root:Epoch[368] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 11.72it/s, avg_epoch_loss=5.41]
INFO:root:Epoch[368] Elapsed time 4.279 seconds
INFO:root:Epoch[368] Evaluation metric 'epoch_loss'=5.405665
INFO:root:Epoch[369] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 12.40it/s, avg_epoch_loss=5.5]
INFO:root:Epoch[369] Elapsed time 4.038 seconds
INFO:root:Epoch[369] Evaluation metric 'epoch_loss'=5.502749
INFO:root:Epoch[370] Learning rate is 5e-05
100%|██████████| 50/50 [00:06<00:00,  8.10it/s, avg_epoch_loss=5.4]
INFO:root:Epoch[370] Elapsed time 6.176 seconds
INFO:root:Epoch[370] Evaluation metric 'epoch_loss'=5.403564
INFO:root:Epoch[371] Learning rate is 5e-05
100%|████

{'MASE': 3.45140916,
 'MSIS': 43.56438325,
 'epochs': 400,
 'sMAPE': 0.03311572,
 'wQuantileLoss[0.5]': 0.0296536,
 'wQuantileLoss[0.9]': 0.01410794}


[{'MASE': 4.00380739,
  'sMAPE': 0.03668879,
  'MSIS': 53.09362333,
  'wQuantileLoss[0.5]': 0.0307505,
  'wQuantileLoss[0.9]': 0.01408046,
  'epochs': 100},
 {'MASE': 3.46475672,
  'sMAPE': 0.03306255,
  'MSIS': 45.44936718,
  'wQuantileLoss[0.5]': 0.02935057,
  'wQuantileLoss[0.9]': 0.01453066,
  'epochs': 150},
 {'MASE': 3.46388428,
  'sMAPE': 0.03304602,
  'MSIS': 45.60462416,
  'wQuantileLoss[0.5]': 0.02936551,
  'wQuantileLoss[0.9]': 0.01459564,
  'epochs': 200},
 {'MASE': 3.55602276,
  'sMAPE': 0.03373353,
  'MSIS': 45.03919041,
  'wQuantileLoss[0.5]': 0.02846558,
  'wQuantileLoss[0.9]': 0.01335697,
  'epochs': 250},
 {'MASE': 3.45378922,
  'sMAPE': 0.03315451,
  'MSIS': 42.52450096,
  'wQuantileLoss[0.5]': 0.02941844,
  'wQuantileLoss[0.9]': 0.01399592,
  'epochs': 300},
 {'MASE': 3.44946768,
  'sMAPE': 0.03311396,
  'MSIS': 42.42848881,
  'wQuantileLoss[0.5]': 0.02932182,
  'wQuantileLoss[0.9]': 0.01399942,
  'epochs': 350},
 {'MASE': 3.45140916,
  'sMAPE': 0.03311572,
  'MSIS'

In [2]:
results44 = []

for i in range(100, 401, 50):
    print("Epochs:", i)
    res = main(data="m4_daily", seed=44, epochs=i, batches=50)
    pprint(res)
    results44.append(res)
    
results44

Epochs: 100


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 11.97it/s, avg_epoch_loss=7.75]
INFO:root:Epoch[0] Elapsed time 4.183 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.750447
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 11.23it/s, avg_epoch_loss=6.5]
INFO:root:Epoch[1] Elapsed time 4.461 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.497455
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 11.15it/s, avg_epoch_loss=6.55]
INFO:root:Epoch[2] Elapsed time 4.490 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.547137
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 11.83it/s, avg_epoch_loss=6.34]
INFO:root:Epoch[3] Elapsed time 4.230 seconds
INFO:

INFO:root:Epoch[73] Learning rate is 0.00025
100%|██████████| 50/50 [00:04<00:00, 10.73it/s, avg_epoch_loss=5.86]
INFO:root:Epoch[73] Elapsed time 4.667 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=5.861712
INFO:root:Epoch[74] Learning rate is 0.00025
100%|██████████| 50/50 [00:04<00:00, 11.23it/s, avg_epoch_loss=5.79]
INFO:root:Epoch[74] Elapsed time 4.459 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=5.789427
INFO:root:Epoch[75] Learning rate is 0.00025
100%|██████████| 50/50 [00:04<00:00, 11.98it/s, avg_epoch_loss=5.77]
INFO:root:Epoch[75] Elapsed time 4.181 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=5.766546
INFO:root:Epoch[76] Learning rate is 0.00025
100%|██████████| 50/50 [00:03<00:00, 12.57it/s, avg_epoch_loss=5.84]
INFO:root:Epoch[76] Elapsed time 3.983 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=5.842619
INFO:root:Epoch[77] Learning rate is 0.00025
100%|██████████| 50/50 [00:04<00:00, 12.31it/s, avg_epoch_loss=5.7]
INF

{'MASE': 3.51571395,
 'MSIS': 49.99930954,
 'epochs': 100,
 'sMAPE': 0.033411,
 'wQuantileLoss[0.5]': 0.02994569,
 'wQuantileLoss[0.9]': 0.01593816}
Epochs: 150


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 14.44it/s, avg_epoch_loss=7.75]
INFO:root:Epoch[0] Elapsed time 3.478 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.750447
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 12.40it/s, avg_epoch_loss=6.5]
INFO:root:Epoch[1] Elapsed time 4.041 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.497455
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 12.11it/s, avg_epoch_loss=6.55]
INFO:root:Epoch[2] Elapsed time 4.133 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.547137
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 12.70it/s, avg_epoch_loss=6.34]
INFO:root:Epoch[3] Elapsed time 3.943 seconds
INFO:

...

100%|██████████| 50/50 [00:04<00:00, 11.53it/s, avg_epoch_loss=5.67]
INFO:root:Epoch[219] Elapsed time 4.344 seconds
INFO:root:Epoch[219] Evaluation metric 'epoch_loss'=5.665740
INFO:root:Epoch[220] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 11.74it/s, avg_epoch_loss=5.59]
INFO:root:Epoch[220] Elapsed time 4.275 seconds
INFO:root:Epoch[220] Evaluation metric 'epoch_loss'=5.585572
INFO:root:Epoch[221] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 11.58it/s, avg_epoch_loss=5.7]
INFO:root:Epoch[221] Elapsed time 4.327 seconds
INFO:root:Epoch[221] Evaluation metric 'epoch_loss'=5.696828
INFO:root:Epoch[222] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 11.54it/s, avg_epoch_loss=5.55]
INFO:root:Epoch[222] Elapsed time 4.339 seconds
INFO:root:Epoch[222] Evaluation metric 'epoch_loss'=5.546974
INFO:root:Epoch[223] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 11.88it/s, avg_epoch_loss=5.68]
INFO:root:Epoch[223] Elapsed time 4.216 seco

INFO:root:Epoch[292] Evaluation metric 'epoch_loss'=5.598640
INFO:root:Epoch[293] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 12.10it/s, avg_epoch_loss=5.53]
INFO:root:Epoch[293] Elapsed time 4.138 seconds
INFO:root:Epoch[293] Evaluation metric 'epoch_loss'=5.532884
INFO:root:Epoch[294] Learning rate is 5e-05
100%|██████████| 50/50 [00:03<00:00, 12.66it/s, avg_epoch_loss=5.62]
INFO:root:Epoch[294] Elapsed time 3.956 seconds
INFO:root:Epoch[294] Evaluation metric 'epoch_loss'=5.619177
INFO:root:Epoch[295] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 12.03it/s, avg_epoch_loss=5.66]
INFO:root:Epoch[295] Elapsed time 4.162 seconds
INFO:root:Epoch[295] Evaluation metric 'epoch_loss'=5.657931
INFO:root:Epoch[296] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 12.04it/s, avg_epoch_loss=5.5]
INFO:root:Epoch[296] Elapsed time 4.160 seconds
INFO:root:Epoch[296] Evaluation metric 'epoch_loss'=5.502007
INFO:root:Epoch[297] Learning rate is 5e-05
100%|███

100%|██████████| 50/50 [00:04<00:00, 11.83it/s, avg_epoch_loss=5.55]
INFO:root:Epoch[366] Elapsed time 4.231 seconds
INFO:root:Epoch[366] Evaluation metric 'epoch_loss'=5.554548
INFO:root:Epoch[367] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 12.37it/s, avg_epoch_loss=5.52]
INFO:root:Epoch[367] Elapsed time 4.047 seconds
INFO:root:Epoch[367] Evaluation metric 'epoch_loss'=5.522796
INFO:root:Epoch[368] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 12.23it/s, avg_epoch_loss=5.6]
INFO:root:Epoch[368] Elapsed time 4.094 seconds
INFO:root:Epoch[368] Evaluation metric 'epoch_loss'=5.602662
INFO:root:Epoch[369] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 12.10it/s, avg_epoch_loss=5.58]
INFO:root:Epoch[369] Elapsed time 4.139 seconds
INFO:root:Epoch[369] Evaluation metric 'epoch_loss'=5.577524
INFO:root:Epoch[370] Learning rate is 5e-05
100%|██████████| 50/50 [00:04<00:00, 12.18it/s, avg_epoch_loss=5.5]
INFO:root:Epoch[370] Elapsed time 4.114 secon

{'MASE': 3.58208824,
 'MSIS': 50.241733,
 'epochs': 400,
 'sMAPE': 0.03377201,
 'wQuantileLoss[0.5]': 0.02832678,
 'wQuantileLoss[0.9]': 0.01350944}


[{'MASE': 3.51571395,
  'sMAPE': 0.033411,
  'MSIS': 49.99930954,
  'wQuantileLoss[0.5]': 0.02994569,
  'wQuantileLoss[0.9]': 0.01593816,
  'epochs': 100},
 {'MASE': 3.38599336,
  'sMAPE': 0.03243939,
  'MSIS': 47.75846253,
  'wQuantileLoss[0.5]': 0.02846284,
  'wQuantileLoss[0.9]': 0.01498526,
  'epochs': 150},
 {'MASE': 3.9249345,
  'sMAPE': 0.03622409,
  'MSIS': 57.02807564,
  'wQuantileLoss[0.5]': 0.03385571,
  'wQuantileLoss[0.9]': 0.01913529,
  'epochs': 200},
 {'MASE': 3.38623901,
  'sMAPE': 0.03239356,
  'MSIS': 47.33066444,
  'wQuantileLoss[0.5]': 0.0286357,
  'wQuantileLoss[0.9]': 0.01502134,
  'epochs': 250},
 {'MASE': 3.38238963,
  'sMAPE': 0.03261771,
  'MSIS': 47.32514592,
  'wQuantileLoss[0.5]': 0.02850876,
  'wQuantileLoss[0.9]': 0.01458218,
  'epochs': 300},
 {'MASE': 3.38442978,
  'sMAPE': 0.03265415,
  'MSIS': 47.49080571,
  'wQuantileLoss[0.5]': 0.02852519,
  'wQuantileLoss[0.9]': 0.0146232,
  'epochs': 350},
 {'MASE': 3.58208824,
  'sMAPE': 0.03377201,
  'MSIS': 50

In [3]:
results = []

for i in range(100, 401, 50):
    print("Epochs:", i)
    res = main(data="m4_daily", seed=42, epochs=i, batches=100)
    pprint(res)
    results.append(res)
    
results

Epochs: 100


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:07<00:00, 13.02it/s, avg_epoch_loss=7.26]
INFO:root:Epoch[0] Elapsed time 7.686 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.258919
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 12.05it/s, avg_epoch_loss=6.44]
INFO:root:Epoch[1] Elapsed time 8.305 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.435382
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 11.80it/s, avg_epoch_loss=6.45]
INFO:root:Epoch[2] Elapsed time 8.482 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.454049
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 12.41it/s, avg_epoch_loss=6.31]
INFO:root:Epoch[3] Elapsed time 8.060 seco

100%|██████████| 100/100 [00:09<00:00, 10.31it/s, avg_epoch_loss=5.41]
INFO:root:Epoch[73] Elapsed time 9.705 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=5.414344
INFO:root:Epoch[74] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 11.89it/s, avg_epoch_loss=5.44]
INFO:root:Epoch[74] Elapsed time 8.412 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=5.444090
INFO:root:Epoch[75] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 12.06it/s, avg_epoch_loss=5.43]
INFO:root:Epoch[75] Elapsed time 8.296 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=5.429206
INFO:root:Epoch[76] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 11.99it/s, avg_epoch_loss=5.5]
INFO:root:Epoch[76] Elapsed time 8.347 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=5.495271
INFO:root:Epoch[77] Learning rate is 0.001
100%|██████████| 100/100 [00:07<00:00, 12.71it/s, avg_epoch_loss=5.41]
INFO:root:Epoch[77] Elapsed time 7.872 seconds

{'MASE': 4.6888278,
 'MSIS': 58.54613315,
 'epochs': 100,
 'sMAPE': 0.04235474,
 'wQuantileLoss[0.5]': 0.03813074,
 'wQuantileLoss[0.9]': 0.01942635}
Epochs: 150


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:07<00:00, 13.47it/s, avg_epoch_loss=7.26]
INFO:root:Epoch[0] Elapsed time 7.432 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.258919
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 12.31it/s, avg_epoch_loss=6.44]
INFO:root:Epoch[1] Elapsed time 8.127 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.435382
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 12.49it/s, avg_epoch_loss=6.45]
INFO:root:Epoch[2] Elapsed time 8.015 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.454049
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 12.09it/s, avg_epoch_loss=6.31]
INFO:root:Epoch[3] Elapsed time 8.281 seco

...

100%|██████████| 100/100 [00:10<00:00,  9.46it/s, avg_epoch_loss=5.38]
INFO:root:Epoch[217] Elapsed time 10.580 seconds
INFO:root:Epoch[217] Evaluation metric 'epoch_loss'=5.377542
INFO:root:Epoch[218] Learning rate is 5e-05
100%|██████████| 100/100 [00:13<00:00,  7.39it/s, avg_epoch_loss=5.36]
INFO:root:Epoch[218] Elapsed time 13.534 seconds
INFO:root:Epoch[218] Evaluation metric 'epoch_loss'=5.364084
INFO:root:Epoch[219] Learning rate is 5e-05
100%|██████████| 100/100 [00:11<00:00,  8.94it/s, avg_epoch_loss=5.38]
INFO:root:Epoch[219] Elapsed time 11.187 seconds
INFO:root:Epoch[219] Evaluation metric 'epoch_loss'=5.381978
INFO:root:Epoch[220] Learning rate is 5e-05
100%|██████████| 100/100 [00:11<00:00,  8.68it/s, avg_epoch_loss=5.41]
INFO:root:Epoch[220] Elapsed time 11.532 seconds
INFO:root:Epoch[220] Evaluation metric 'epoch_loss'=5.410843
INFO:root:Epoch[221] Learning rate is 5e-05
100%|██████████| 100/100 [00:14<00:00,  6.74it/s, avg_epoch_loss=5.32]
INFO:root:Epoch[221] Elapsed 

INFO:root:Epoch[289] Evaluation metric 'epoch_loss'=5.368700
INFO:root:Epoch[290] Learning rate is 5e-05
100%|██████████| 100/100 [00:11<00:00,  8.54it/s, avg_epoch_loss=5.38]
INFO:root:Epoch[290] Elapsed time 11.713 seconds
INFO:root:Epoch[290] Evaluation metric 'epoch_loss'=5.382119
INFO:root:Epoch[291] Learning rate is 5e-05
100%|██████████| 100/100 [00:11<00:00,  8.64it/s, avg_epoch_loss=5.39]
INFO:root:Epoch[291] Elapsed time 11.584 seconds
INFO:root:Epoch[291] Evaluation metric 'epoch_loss'=5.388878
INFO:root:Epoch[292] Learning rate is 5e-05
100%|██████████| 100/100 [00:12<00:00,  8.31it/s, avg_epoch_loss=5.36]
INFO:root:Epoch[292] Elapsed time 12.046 seconds
INFO:root:Epoch[292] Evaluation metric 'epoch_loss'=5.359219
INFO:root:Epoch[293] Learning rate is 5e-05
100%|██████████| 100/100 [00:11<00:00,  8.66it/s, avg_epoch_loss=5.36]
INFO:root:Epoch[293] Elapsed time 11.558 seconds
INFO:root:Epoch[293] Evaluation metric 'epoch_loss'=5.360035
INFO:root:Epoch[294] Learning rate is 5

100%|██████████| 100/100 [00:08<00:00, 11.32it/s, avg_epoch_loss=5.28]
INFO:root:Epoch[362] Elapsed time 8.839 seconds
INFO:root:Epoch[362] Evaluation metric 'epoch_loss'=5.283069
INFO:root:Epoch[363] Learning rate is 5e-05
100%|██████████| 100/100 [00:09<00:00, 10.79it/s, avg_epoch_loss=5.42]
INFO:root:Epoch[363] Elapsed time 9.275 seconds
INFO:root:Epoch[363] Evaluation metric 'epoch_loss'=5.416227
INFO:root:Epoch[364] Learning rate is 5e-05
100%|██████████| 100/100 [00:10<00:00,  9.81it/s, avg_epoch_loss=5.38]
INFO:root:Epoch[364] Elapsed time 10.206 seconds
INFO:root:Epoch[364] Evaluation metric 'epoch_loss'=5.383461
INFO:root:Epoch[365] Learning rate is 5e-05
100%|██████████| 100/100 [00:10<00:00,  9.40it/s, avg_epoch_loss=5.39]
INFO:root:Epoch[365] Elapsed time 10.651 seconds
INFO:root:Epoch[365] Evaluation metric 'epoch_loss'=5.385748
INFO:root:Epoch[366] Learning rate is 5e-05
100%|██████████| 100/100 [00:10<00:00,  9.36it/s, avg_epoch_loss=5.31]
INFO:root:Epoch[366] Elapsed ti

{'MASE': 3.37004807,
 'MSIS': 40.71273682,
 'epochs': 400,
 'sMAPE': 0.03205514,
 'wQuantileLoss[0.5]': 0.02758192,
 'wQuantileLoss[0.9]': 0.01278391}


[{'MASE': 4.6888278,
  'sMAPE': 0.04235474,
  'MSIS': 58.54613315,
  'wQuantileLoss[0.5]': 0.03813074,
  'wQuantileLoss[0.9]': 0.01942635,
  'epochs': 100},
 {'MASE': 3.34701839,
  'sMAPE': 0.03185894,
  'MSIS': 40.38508533,
  'wQuantileLoss[0.5]': 0.02774475,
  'wQuantileLoss[0.9]': 0.01301838,
  'epochs': 150},
 {'MASE': 3.34129322,
  'sMAPE': 0.03175431,
  'MSIS': 40.38673493,
  'wQuantileLoss[0.5]': 0.02770024,
  'wQuantileLoss[0.9]': 0.01305936,
  'epochs': 200},
 {'MASE': 3.607182,
  'sMAPE': 0.03346469,
  'MSIS': 42.24145034,
  'wQuantileLoss[0.5]': 0.03064602,
  'wQuantileLoss[0.9]': 0.01383277,
  'epochs': 250},
 {'MASE': 3.37422146,
  'sMAPE': 0.03209932,
  'MSIS': 40.7909502,
  'wQuantileLoss[0.5]': 0.0276275,
  'wQuantileLoss[0.9]': 0.0128443,
  'epochs': 300},
 {'MASE': 3.38032501,
  'sMAPE': 0.03213667,
  'MSIS': 40.94456104,
  'wQuantileLoss[0.5]': 0.02768916,
  'wQuantileLoss[0.9]': 0.01282163,
  'epochs': 350},
 {'MASE': 3.37004807,
  'sMAPE': 0.03205514,
  'MSIS': 40.

In [2]:
results = []

for i in range(100, 401, 50):
    print("Epochs:", i)
    res = main(data="m4_daily", seed=43, epochs=i, batches=100)
    pprint(res)
    results.append(res)
    
results

Epochs: 100


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:09<00:00, 10.41it/s, avg_epoch_loss=6.94]
INFO:root:Epoch[0] Elapsed time 9.612 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.943348
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:09<00:00, 11.07it/s, avg_epoch_loss=6.54]
INFO:root:Epoch[1] Elapsed time 9.042 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.544661
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:12<00:00,  8.24it/s, avg_epoch_loss=6.43]
INFO:root:Epoch[2] Elapsed time 12.149 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.434662
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:12<00:00,  7.84it/s, avg_epoch_loss=6.28]
INFO:root:Epoch[3] Elapsed time 12.754 se

INFO:root:Epoch[73] Learning rate is 0.0005
100%|██████████| 100/100 [00:08<00:00, 12.04it/s, avg_epoch_loss=5.45]
INFO:root:Epoch[73] Elapsed time 8.310 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=5.453764
INFO:root:Epoch[74] Learning rate is 0.0005
100%|██████████| 100/100 [00:08<00:00, 12.46it/s, avg_epoch_loss=5.43]
INFO:root:Epoch[74] Elapsed time 8.035 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=5.425285
INFO:root:Epoch[75] Learning rate is 0.0005
100%|██████████| 100/100 [00:08<00:00, 12.13it/s, avg_epoch_loss=5.46]
INFO:root:Epoch[75] Elapsed time 8.248 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=5.456648
INFO:root:Epoch[76] Learning rate is 0.0005
100%|██████████| 100/100 [00:08<00:00, 12.40it/s, avg_epoch_loss=5.42]
INFO:root:Epoch[76] Elapsed time 8.072 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=5.422869
INFO:root:Epoch[77] Learning rate is 0.0005
100%|██████████| 100/100 [00:08<00:00, 12.37it/s, avg_epoch_loss=5.5

{'MASE': 3.76198041,
 'MSIS': 42.13462873,
 'epochs': 100,
 'sMAPE': 0.03513189,
 'wQuantileLoss[0.5]': 0.03183971,
 'wQuantileLoss[0.9]': 0.01504344}
Epochs: 150


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:12<00:00,  8.21it/s, avg_epoch_loss=6.94]
INFO:root:Epoch[0] Elapsed time 12.183 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.943348
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:16<00:00,  6.10it/s, avg_epoch_loss=6.54]
INFO:root:Epoch[1] Elapsed time 16.396 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.544661
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:09<00:00, 10.08it/s, avg_epoch_loss=6.43]
INFO:root:Epoch[2] Elapsed time 9.924 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.434662
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:10<00:00,  9.94it/s, avg_epoch_loss=6.28]
INFO:root:Epoch[3] Elapsed time 10.069 s

...

100%|██████████| 100/100 [00:10<00:00,  9.26it/s, avg_epoch_loss=5.43]
INFO:root:Epoch[253] Elapsed time 10.811 seconds
INFO:root:Epoch[253] Evaluation metric 'epoch_loss'=5.426982
INFO:root:Epoch[254] Learning rate is 5e-05
100%|██████████| 100/100 [00:11<00:00,  8.88it/s, avg_epoch_loss=5.4]
INFO:root:Epoch[254] Elapsed time 11.266 seconds
INFO:root:Epoch[254] Evaluation metric 'epoch_loss'=5.402783
INFO:root:Epoch[255] Learning rate is 5e-05
100%|██████████| 100/100 [00:11<00:00,  8.98it/s, avg_epoch_loss=5.39]
INFO:root:Epoch[255] Elapsed time 11.143 seconds
INFO:root:Epoch[255] Evaluation metric 'epoch_loss'=5.390460
INFO:root:Epoch[256] Learning rate is 5e-05
100%|██████████| 100/100 [00:10<00:00,  9.13it/s, avg_epoch_loss=5.36]
INFO:root:Epoch[256] Elapsed time 10.960 seconds
INFO:root:Epoch[256] Evaluation metric 'epoch_loss'=5.361468
INFO:root:Epoch[257] Learning rate is 5e-05
100%|██████████| 100/100 [00:10<00:00,  9.33it/s, avg_epoch_loss=5.43]
INFO:root:Epoch[257] Elapsed t

INFO:root:Epoch[326] Learning rate is 5e-05
100%|██████████| 100/100 [00:08<00:00, 11.80it/s, avg_epoch_loss=5.35]
INFO:root:Epoch[326] Elapsed time 8.486 seconds
INFO:root:Epoch[326] Evaluation metric 'epoch_loss'=5.351460
INFO:root:Epoch[327] Learning rate is 5e-05
100%|██████████| 100/100 [00:08<00:00, 11.60it/s, avg_epoch_loss=5.34]
INFO:root:Epoch[327] Elapsed time 8.624 seconds
INFO:root:Epoch[327] Evaluation metric 'epoch_loss'=5.344435
INFO:root:Epoch[328] Learning rate is 5e-05
100%|██████████| 100/100 [00:08<00:00, 11.83it/s, avg_epoch_loss=5.36]
INFO:root:Epoch[328] Elapsed time 8.458 seconds
INFO:root:Epoch[328] Evaluation metric 'epoch_loss'=5.360654
INFO:root:Epoch[329] Learning rate is 5e-05
100%|██████████| 100/100 [00:08<00:00, 11.68it/s, avg_epoch_loss=5.45]
INFO:root:Epoch[329] Elapsed time 8.573 seconds
INFO:root:Epoch[329] Evaluation metric 'epoch_loss'=5.445871
INFO:root:Epoch[330] Learning rate is 5e-05
100%|██████████| 100/100 [00:08<00:00, 12.00it/s, avg_epoch_

INFO:root:Epoch[398] Evaluation metric 'epoch_loss'=5.385466
INFO:root:Epoch[399] Learning rate is 5e-05
100%|██████████| 100/100 [00:10<00:00,  9.42it/s, avg_epoch_loss=5.39]
INFO:root:Epoch[399] Elapsed time 10.622 seconds
INFO:root:Epoch[399] Evaluation metric 'epoch_loss'=5.388206
INFO:root:Loading parameters from best epoch (190)
INFO:root:Final loss: 5.307220115661621 (occurred at epoch 190)
INFO:root:End model training
Running evaluation: 100%|██████████| 4227/4227 [06:33<00:00, 10.74it/s]


{'MASE': 3.53992429,
 'MSIS': 43.04474713,
 'epochs': 400,
 'sMAPE': 0.03322258,
 'wQuantileLoss[0.5]': 0.03089263,
 'wQuantileLoss[0.9]': 0.0144801}


[{'MASE': 3.76198041,
  'sMAPE': 0.03513189,
  'MSIS': 42.13462873,
  'wQuantileLoss[0.5]': 0.03183971,
  'wQuantileLoss[0.9]': 0.01504344,
  'epochs': 100},
 {'MASE': 3.809087,
  'sMAPE': 0.03519001,
  'MSIS': 46.15768663,
  'wQuantileLoss[0.5]': 0.03329824,
  'wQuantileLoss[0.9]': 0.01593606,
  'epochs': 150},
 {'MASE': 3.54183074,
  'sMAPE': 0.03324842,
  'MSIS': 43.06691573,
  'wQuantileLoss[0.5]': 0.03086203,
  'wQuantileLoss[0.9]': 0.01445214,
  'epochs': 200},
 {'MASE': 3.53705613,
  'sMAPE': 0.03320447,
  'MSIS': 42.80685746,
  'wQuantileLoss[0.5]': 0.03080833,
  'wQuantileLoss[0.9]': 0.01434419,
  'epochs': 250},
 {'MASE': 3.53236466,
  'sMAPE': 0.03318516,
  'MSIS': 42.98269824,
  'wQuantileLoss[0.5]': 0.03079269,
  'wQuantileLoss[0.9]': 0.01442702,
  'epochs': 300},
 {'MASE': 3.53595053,
  'sMAPE': 0.0332005,
  'MSIS': 42.99588989,
  'wQuantileLoss[0.5]': 0.03082195,
  'wQuantileLoss[0.9]': 0.01440264,
  'epochs': 350},
 {'MASE': 3.53992429,
  'sMAPE': 0.03322258,
  'MSIS': 

In [2]:
res = main(data="m4_daily", seed=44, epochs=100, batches=100)
pprint(res)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 11.93it/s, avg_epoch_loss=7.12]
INFO:root:Epoch[0] Elapsed time 8.391 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.123951
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 11.89it/s, avg_epoch_loss=6.44]
INFO:root:Epoch[1] Elapsed time 8.422 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.443972
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 11.51it/s, avg_epoch_loss=6.42]
INFO:root:Epoch[2] Elapsed time 8.696 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.415508
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 11.84it/s, avg_epoch_loss=6.35]
INFO:root:Epoch[3] Elapsed time 8.450 seco

INFO:root:Epoch[73] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 11.66it/s, avg_epoch_loss=5.46]
INFO:root:Epoch[73] Elapsed time 8.586 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=5.464562
INFO:root:Epoch[74] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 11.37it/s, avg_epoch_loss=5.46]
INFO:root:Epoch[74] Elapsed time 8.804 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=5.463960
INFO:root:Epoch[75] Learning rate is 0.001
100%|██████████| 100/100 [00:09<00:00, 10.67it/s, avg_epoch_loss=5.42]
INFO:root:Epoch[75] Elapsed time 9.381 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=5.416290
INFO:root:Epoch[76] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 11.59it/s, avg_epoch_loss=5.49]
INFO:root:Epoch[76] Elapsed time 8.636 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=5.492530
INFO:root:Epoch[77] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 11.42it/s, avg_epoch_loss=5.48]
IN

{'MASE': 3.74909462,
 'MSIS': 44.04607147,
 'epochs': 100,
 'sMAPE': 0.03473306,
 'wQuantileLoss[0.5]': 0.03155759,
 'wQuantileLoss[0.9]': 0.01412131}


In [6]:
res = main(data="m4_daily", seed=44, epochs=150, batches=100)
pprint(res)

In [ ]:
res = main(data="m4_daily", seed=44, epochs=200, batches=100)
pprint(res)

In [ ]:
res = main(data="m4_daily", seed=44, epochs=250, batches=100)
pprint(res)

In [ ]:
res = main(data="m4_daily", seed=44, epochs=300, batches=100)
pprint(res)

In [ ]:
res = main(data="m4_daily", seed=44, epochs=350, batches=100)
pprint(res)

In [ ]:
res = main(data="m4_daily", seed=44, epochs=400, batches=100)
pprint(res)

In [4]:
res = main(data="m4_daily", seed=42, epochs=100, batches=200)
pprint(res)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 200/200 [00:27<00:00,  7.32it/s, avg_epoch_loss=6.85]
INFO:root:Epoch[0] Elapsed time 27.336 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.847151
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:25<00:00,  7.96it/s, avg_epoch_loss=6.38]
INFO:root:Epoch[1] Elapsed time 25.147 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.382643
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:25<00:00,  7.94it/s, avg_epoch_loss=6.24]
INFO:root:Epoch[2] Elapsed time 25.191 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.242576
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:21<00:00,  9.49it/s, avg_epoch_loss=6.13]
INFO:root:Epoch[3] Elapsed time 21.091 

INFO:root:Epoch[72] Elapsed time 18.369 seconds
INFO:root:Epoch[72] Evaluation metric 'epoch_loss'=5.394916
INFO:root:Epoch[73] Learning rate is 0.0005
100%|██████████| 200/200 [00:17<00:00, 11.16it/s, avg_epoch_loss=5.41]
INFO:root:Epoch[73] Elapsed time 17.932 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=5.414899
INFO:root:Epoch[74] Learning rate is 0.0005
100%|██████████| 200/200 [00:17<00:00, 11.42it/s, avg_epoch_loss=5.38]
INFO:root:Epoch[74] Elapsed time 17.519 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=5.376001
INFO:root:Epoch[75] Learning rate is 0.0005
100%|██████████| 200/200 [00:17<00:00, 11.62it/s, avg_epoch_loss=5.42]
INFO:root:Epoch[75] Elapsed time 17.227 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=5.415727
INFO:root:Epoch[76] Learning rate is 0.0005
100%|██████████| 200/200 [00:18<00:00, 10.88it/s, avg_epoch_loss=5.39]
INFO:root:Epoch[76] Elapsed time 18.396 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=5.386506


{'MASE': 5.4063357,
 'MSIS': 77.86403918,
 'epochs': 100,
 'sMAPE': 0.04754383,
 'wQuantileLoss[0.5]': 0.04598875,
 'wQuantileLoss[0.9]': 0.02633989}


In [2]:
res = main(data="m4_daily", seed=42, epochs=150, batches=200)
pprint(res)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 200/200 [00:21<00:00,  9.20it/s, avg_epoch_loss=6.85]
INFO:root:Epoch[0] Elapsed time 21.751 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.847151
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:23<00:00,  8.62it/s, avg_epoch_loss=6.38]
INFO:root:Epoch[1] Elapsed time 23.202 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.382643
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:33<00:00,  5.94it/s, avg_epoch_loss=6.24]
INFO:root:Epoch[2] Elapsed time 33.702 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.242576
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:41<00:00,  4.82it/s, avg_epoch_loss=6.13]
INFO:root:Epoch[3] Elapsed time 41.465 

INFO:root:Epoch[36] Learning rate is 0.001
100%|██████████| 200/200 [00:16<00:00, 12.05it/s, avg_epoch_loss=5.47]
INFO:root:Epoch[36] Elapsed time 16.597 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=5.466517
INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 200/200 [00:16<00:00, 12.22it/s, avg_epoch_loss=5.44]
INFO:root:Epoch[37] Elapsed time 16.369 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=5.436648
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 200/200 [00:16<00:00, 12.26it/s, avg_epoch_loss=5.45]
INFO:root:Epoch[38] Elapsed time 16.322 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=5.453099
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 200/200 [00:16<00:00, 12.29it/s, avg_epoch_loss=5.46]
INFO:root:Epoch[39] Elapsed time 16.282 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=5.461915
INFO:root:Epoch[40] Learning rate is 0.001
100%|██████████| 200/200 [00:16<00:00, 12.22it/s, avg_epoch_loss=5.47

INFO:root:Epoch[72] Elapsed time 16.240 seconds
INFO:root:Epoch[72] Evaluation metric 'epoch_loss'=5.394916
INFO:root:Epoch[73] Learning rate is 0.0005
100%|██████████| 200/200 [00:16<00:00, 12.34it/s, avg_epoch_loss=5.41]
INFO:root:Epoch[73] Elapsed time 16.213 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=5.414899
INFO:root:Epoch[74] Learning rate is 0.0005
100%|██████████| 200/200 [00:16<00:00, 12.40it/s, avg_epoch_loss=5.38]
INFO:root:Epoch[74] Elapsed time 16.130 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=5.376001
INFO:root:Epoch[75] Learning rate is 0.0005
100%|██████████| 200/200 [00:16<00:00, 12.50it/s, avg_epoch_loss=5.42]
INFO:root:Epoch[75] Elapsed time 16.011 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=5.415727
INFO:root:Epoch[76] Learning rate is 0.0005
100%|██████████| 200/200 [00:16<00:00, 12.41it/s, avg_epoch_loss=5.39]
INFO:root:Epoch[76] Elapsed time 16.123 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=5.386506


INFO:root:Epoch[108] Evaluation metric 'epoch_loss'=5.375312
INFO:root:Epoch[109] Learning rate is 0.00025
100%|██████████| 200/200 [00:24<00:00,  8.28it/s, avg_epoch_loss=5.35]
INFO:root:Epoch[109] Elapsed time 24.166 seconds
INFO:root:Epoch[109] Evaluation metric 'epoch_loss'=5.353056
INFO:root:Epoch[110] Learning rate is 0.00025
100%|██████████| 200/200 [00:21<00:00,  9.33it/s, avg_epoch_loss=5.34]
INFO:root:Epoch[110] Elapsed time 21.434 seconds
INFO:root:Epoch[110] Evaluation metric 'epoch_loss'=5.344916
INFO:root:Epoch[111] Learning rate is 0.00025
100%|██████████| 200/200 [00:21<00:00,  9.37it/s, avg_epoch_loss=5.33]
INFO:root:Epoch[111] Elapsed time 21.364 seconds
INFO:root:Epoch[111] Evaluation metric 'epoch_loss'=5.334667
INFO:root:Epoch[112] Learning rate is 0.00025
100%|██████████| 200/200 [00:20<00:00,  9.85it/s, avg_epoch_loss=5.37]
INFO:root:Epoch[112] Elapsed time 20.315 seconds
INFO:root:Epoch[112] Evaluation metric 'epoch_loss'=5.371374
INFO:root:Epoch[113] Learning r

100%|██████████| 200/200 [00:20<00:00,  9.61it/s, avg_epoch_loss=5.32]
INFO:root:Epoch[144] Elapsed time 20.822 seconds
INFO:root:Epoch[144] Evaluation metric 'epoch_loss'=5.318786
INFO:root:Epoch[145] Learning rate is 6.25e-05
100%|██████████| 200/200 [00:36<00:00,  5.46it/s, avg_epoch_loss=5.36]
INFO:root:Epoch[145] Elapsed time 36.657 seconds
INFO:root:Epoch[145] Evaluation metric 'epoch_loss'=5.360012
INFO:root:Epoch[146] Learning rate is 6.25e-05
100%|██████████| 200/200 [00:18<00:00, 10.77it/s, avg_epoch_loss=5.33]
INFO:root:Epoch[146] Elapsed time 18.579 seconds
INFO:root:Epoch[146] Evaluation metric 'epoch_loss'=5.332261
INFO:root:Epoch[147] Learning rate is 6.25e-05
100%|██████████| 200/200 [00:17<00:00, 11.28it/s, avg_epoch_loss=5.36]
INFO:root:Epoch[147] Elapsed time 17.730 seconds
INFO:root:Epoch[147] Evaluation metric 'epoch_loss'=5.357236
INFO:root:Epoch[148] Learning rate is 6.25e-05
100%|██████████| 200/200 [00:23<00:00,  8.54it/s, avg_epoch_loss=5.34]
INFO:root:Epoch[1

{'MASE': 3.31045227,
 'MSIS': 40.79797344,
 'epochs': 150,
 'sMAPE': 0.0316434,
 'wQuantileLoss[0.5]': 0.02787393,
 'wQuantileLoss[0.9]': 0.01310215}


In [3]:
res = main(data="m4_daily", seed=42, epochs=200, batches=200)
pprint(res)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 200/200 [00:19<00:00, 10.37it/s, avg_epoch_loss=6.85]
INFO:root:Epoch[0] Elapsed time 19.292 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.847151
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:27<00:00,  7.17it/s, avg_epoch_loss=6.38]
INFO:root:Epoch[1] Elapsed time 27.896 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.382643
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:17<00:00, 11.35it/s, avg_epoch_loss=6.24]
INFO:root:Epoch[2] Elapsed time 17.634 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.242576
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:17<00:00, 11.48it/s, avg_epoch_loss=6.13]
INFO:root:Epoch[3] Elapsed time 17.428 

INFO:root:Epoch[36] Learning rate is 0.001
100%|██████████| 200/200 [00:18<00:00, 10.90it/s, avg_epoch_loss=5.47]
INFO:root:Epoch[36] Elapsed time 18.349 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=5.466517
INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 200/200 [00:18<00:00, 10.82it/s, avg_epoch_loss=5.44]
INFO:root:Epoch[37] Elapsed time 18.495 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=5.436648
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 200/200 [00:18<00:00, 10.80it/s, avg_epoch_loss=5.45]
INFO:root:Epoch[38] Elapsed time 18.527 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=5.453099
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 200/200 [00:17<00:00, 11.46it/s, avg_epoch_loss=5.46]
INFO:root:Epoch[39] Elapsed time 17.460 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=5.461915
INFO:root:Epoch[40] Learning rate is 0.001
100%|██████████| 200/200 [00:18<00:00, 10.96it/s, avg_epoch_loss=5.47

INFO:root:Epoch[72] Elapsed time 18.676 seconds
INFO:root:Epoch[72] Evaluation metric 'epoch_loss'=5.394916
INFO:root:Epoch[73] Learning rate is 0.0005
100%|██████████| 200/200 [00:18<00:00, 11.09it/s, avg_epoch_loss=5.41]
INFO:root:Epoch[73] Elapsed time 18.038 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=5.414899
INFO:root:Epoch[74] Learning rate is 0.0005
100%|██████████| 200/200 [00:18<00:00, 10.73it/s, avg_epoch_loss=5.38]
INFO:root:Epoch[74] Elapsed time 18.652 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=5.376001
INFO:root:Epoch[75] Learning rate is 0.0005
100%|██████████| 200/200 [00:18<00:00, 10.77it/s, avg_epoch_loss=5.42]
INFO:root:Epoch[75] Elapsed time 18.579 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=5.415727
INFO:root:Epoch[76] Learning rate is 0.0005
100%|██████████| 200/200 [00:19<00:00, 10.35it/s, avg_epoch_loss=5.39]
INFO:root:Epoch[76] Elapsed time 19.331 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=5.386506


INFO:root:Epoch[108] Evaluation metric 'epoch_loss'=5.375312
INFO:root:Epoch[109] Learning rate is 0.00025
100%|██████████| 200/200 [00:18<00:00, 10.79it/s, avg_epoch_loss=5.35]
INFO:root:Epoch[109] Elapsed time 18.541 seconds
INFO:root:Epoch[109] Evaluation metric 'epoch_loss'=5.353056
INFO:root:Epoch[110] Learning rate is 0.00025
100%|██████████| 200/200 [00:20<00:00,  9.61it/s, avg_epoch_loss=5.34]
INFO:root:Epoch[110] Elapsed time 20.809 seconds
INFO:root:Epoch[110] Evaluation metric 'epoch_loss'=5.344916
INFO:root:Epoch[111] Learning rate is 0.00025
100%|██████████| 200/200 [00:19<00:00, 10.40it/s, avg_epoch_loss=5.33]
INFO:root:Epoch[111] Elapsed time 19.243 seconds
INFO:root:Epoch[111] Evaluation metric 'epoch_loss'=5.334667
INFO:root:Epoch[112] Learning rate is 0.00025
100%|██████████| 200/200 [00:42<00:00,  4.75it/s, avg_epoch_loss=5.37]
INFO:root:Epoch[112] Elapsed time 42.139 seconds
INFO:root:Epoch[112] Evaluation metric 'epoch_loss'=5.371374
INFO:root:Epoch[113] Learning r

100%|██████████| 200/200 [00:27<00:00,  7.20it/s, avg_epoch_loss=5.32]
INFO:root:Epoch[144] Elapsed time 27.779 seconds
INFO:root:Epoch[144] Evaluation metric 'epoch_loss'=5.318786
INFO:root:Epoch[145] Learning rate is 6.25e-05
100%|██████████| 200/200 [00:23<00:00,  8.44it/s, avg_epoch_loss=5.36]
INFO:root:Epoch[145] Elapsed time 23.696 seconds
INFO:root:Epoch[145] Evaluation metric 'epoch_loss'=5.360012
INFO:root:Epoch[146] Learning rate is 6.25e-05
100%|██████████| 200/200 [00:27<00:00,  7.33it/s, avg_epoch_loss=5.33]
INFO:root:Epoch[146] Elapsed time 27.294 seconds
INFO:root:Epoch[146] Evaluation metric 'epoch_loss'=5.332261
INFO:root:Epoch[147] Learning rate is 6.25e-05
100%|██████████| 200/200 [00:25<00:00,  7.72it/s, avg_epoch_loss=5.36]
INFO:root:Epoch[147] Elapsed time 25.910 seconds
INFO:root:Epoch[147] Evaluation metric 'epoch_loss'=5.357236
INFO:root:Epoch[148] Learning rate is 6.25e-05
100%|██████████| 200/200 [00:18<00:00, 10.60it/s, avg_epoch_loss=5.34]
INFO:root:Epoch[1

100%|██████████| 200/200 [00:17<00:00, 11.31it/s, avg_epoch_loss=5.34]
INFO:root:Epoch[180] Elapsed time 17.687 seconds
INFO:root:Epoch[180] Evaluation metric 'epoch_loss'=5.338457
INFO:root:Epoch[181] Learning rate is 5e-05
100%|██████████| 200/200 [00:18<00:00, 10.60it/s, avg_epoch_loss=5.33]
INFO:root:Epoch[181] Elapsed time 18.867 seconds
INFO:root:Epoch[181] Evaluation metric 'epoch_loss'=5.327507
INFO:root:Epoch[182] Learning rate is 5e-05
100%|██████████| 200/200 [00:20<00:00,  9.66it/s, avg_epoch_loss=5.36]
INFO:root:Epoch[182] Elapsed time 20.705 seconds
INFO:root:Epoch[182] Evaluation metric 'epoch_loss'=5.361690
INFO:root:Epoch[183] Learning rate is 5e-05
100%|██████████| 200/200 [00:24<00:00,  8.20it/s, avg_epoch_loss=5.31]
INFO:root:Epoch[183] Elapsed time 24.400 seconds
INFO:root:Epoch[183] Evaluation metric 'epoch_loss'=5.313974
INFO:root:Epoch[184] Learning rate is 5e-05
100%|██████████| 200/200 [00:20<00:00,  9.96it/s, avg_epoch_loss=5.38]
INFO:root:Epoch[184] Elapsed 

{'MASE': 3.52237985,
 'MSIS': 41.86274029,
 'epochs': 200,
 'sMAPE': 0.03313088,
 'wQuantileLoss[0.5]': 0.02980716,
 'wQuantileLoss[0.9]': 0.01329117}


In [4]:
res = main(data="m4_daily", seed=42, epochs=250, batches=200)
pprint(res)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 200/200 [00:24<00:00,  8.23it/s, avg_epoch_loss=6.85]
INFO:root:Epoch[0] Elapsed time 24.307 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.847151
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:22<00:00,  8.95it/s, avg_epoch_loss=6.38]
INFO:root:Epoch[1] Elapsed time 22.346 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.382643
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:23<00:00,  8.66it/s, avg_epoch_loss=6.24]
INFO:root:Epoch[2] Elapsed time 23.102 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.242576
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:21<00:00,  9.31it/s, avg_epoch_loss=6.13]
INFO:root:Epoch[3] Elapsed time 21.498 

INFO:root:Epoch[36] Learning rate is 0.001
100%|██████████| 200/200 [00:17<00:00, 11.38it/s, avg_epoch_loss=5.47]
INFO:root:Epoch[36] Elapsed time 17.577 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=5.466517
INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 200/200 [00:17<00:00, 11.47it/s, avg_epoch_loss=5.44]
INFO:root:Epoch[37] Elapsed time 17.443 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=5.436648
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 200/200 [00:17<00:00, 11.38it/s, avg_epoch_loss=5.45]
INFO:root:Epoch[38] Elapsed time 17.586 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=5.453099
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 200/200 [00:17<00:00, 11.53it/s, avg_epoch_loss=5.46]
INFO:root:Epoch[39] Elapsed time 17.354 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=5.461915
INFO:root:Epoch[40] Learning rate is 0.001
100%|██████████| 200/200 [00:18<00:00, 10.87it/s, avg_epoch_loss=5.47

INFO:root:Epoch[72] Elapsed time 18.232 seconds
INFO:root:Epoch[72] Evaluation metric 'epoch_loss'=5.394916
INFO:root:Epoch[73] Learning rate is 0.0005
100%|██████████| 200/200 [00:17<00:00, 11.47it/s, avg_epoch_loss=5.41]
INFO:root:Epoch[73] Elapsed time 17.438 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=5.414899
INFO:root:Epoch[74] Learning rate is 0.0005
100%|██████████| 200/200 [00:17<00:00, 11.71it/s, avg_epoch_loss=5.38]
INFO:root:Epoch[74] Elapsed time 17.092 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=5.376001
INFO:root:Epoch[75] Learning rate is 0.0005
100%|██████████| 200/200 [00:17<00:00, 11.53it/s, avg_epoch_loss=5.42]
INFO:root:Epoch[75] Elapsed time 17.347 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=5.415727
INFO:root:Epoch[76] Learning rate is 0.0005
100%|██████████| 200/200 [00:17<00:00, 11.41it/s, avg_epoch_loss=5.39]
INFO:root:Epoch[76] Elapsed time 17.538 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=5.386506


INFO:root:Epoch[108] Evaluation metric 'epoch_loss'=5.375312
INFO:root:Epoch[109] Learning rate is 0.00025
100%|██████████| 200/200 [00:17<00:00, 11.16it/s, avg_epoch_loss=5.35]
INFO:root:Epoch[109] Elapsed time 17.924 seconds
INFO:root:Epoch[109] Evaluation metric 'epoch_loss'=5.353056
INFO:root:Epoch[110] Learning rate is 0.00025
100%|██████████| 200/200 [00:18<00:00, 10.95it/s, avg_epoch_loss=5.34]
INFO:root:Epoch[110] Elapsed time 18.272 seconds
INFO:root:Epoch[110] Evaluation metric 'epoch_loss'=5.344916
INFO:root:Epoch[111] Learning rate is 0.00025
100%|██████████| 200/200 [00:17<00:00, 11.19it/s, avg_epoch_loss=5.33]
INFO:root:Epoch[111] Elapsed time 17.874 seconds
INFO:root:Epoch[111] Evaluation metric 'epoch_loss'=5.334667
INFO:root:Epoch[112] Learning rate is 0.00025
100%|██████████| 200/200 [00:17<00:00, 11.36it/s, avg_epoch_loss=5.37]
INFO:root:Epoch[112] Elapsed time 17.617 seconds
INFO:root:Epoch[112] Evaluation metric 'epoch_loss'=5.371374
INFO:root:Epoch[113] Learning r

100%|██████████| 200/200 [00:21<00:00,  9.29it/s, avg_epoch_loss=5.32]
INFO:root:Epoch[144] Elapsed time 21.545 seconds
INFO:root:Epoch[144] Evaluation metric 'epoch_loss'=5.318786
INFO:root:Epoch[145] Learning rate is 6.25e-05
100%|██████████| 200/200 [00:18<00:00, 10.79it/s, avg_epoch_loss=5.36]
INFO:root:Epoch[145] Elapsed time 18.537 seconds
INFO:root:Epoch[145] Evaluation metric 'epoch_loss'=5.360012
INFO:root:Epoch[146] Learning rate is 6.25e-05
100%|██████████| 200/200 [00:17<00:00, 11.20it/s, avg_epoch_loss=5.33]
INFO:root:Epoch[146] Elapsed time 17.870 seconds
INFO:root:Epoch[146] Evaluation metric 'epoch_loss'=5.332261
INFO:root:Epoch[147] Learning rate is 6.25e-05
100%|██████████| 200/200 [00:17<00:00, 11.28it/s, avg_epoch_loss=5.36]
INFO:root:Epoch[147] Elapsed time 17.736 seconds
INFO:root:Epoch[147] Evaluation metric 'epoch_loss'=5.357236
INFO:root:Epoch[148] Learning rate is 6.25e-05
100%|██████████| 200/200 [00:18<00:00, 11.09it/s, avg_epoch_loss=5.34]
INFO:root:Epoch[1

100%|██████████| 200/200 [00:19<00:00, 10.17it/s, avg_epoch_loss=5.34]
INFO:root:Epoch[180] Elapsed time 19.677 seconds
INFO:root:Epoch[180] Evaluation metric 'epoch_loss'=5.338457
INFO:root:Epoch[181] Learning rate is 5e-05
100%|██████████| 200/200 [00:20<00:00,  9.67it/s, avg_epoch_loss=5.33]
INFO:root:Epoch[181] Elapsed time 20.693 seconds
INFO:root:Epoch[181] Evaluation metric 'epoch_loss'=5.327507
INFO:root:Epoch[182] Learning rate is 5e-05
100%|██████████| 200/200 [00:19<00:00, 10.51it/s, avg_epoch_loss=5.36]
INFO:root:Epoch[182] Elapsed time 19.045 seconds
INFO:root:Epoch[182] Evaluation metric 'epoch_loss'=5.361690
INFO:root:Epoch[183] Learning rate is 5e-05
100%|██████████| 200/200 [00:17<00:00, 11.25it/s, avg_epoch_loss=5.31]
INFO:root:Epoch[183] Elapsed time 17.786 seconds
INFO:root:Epoch[183] Evaluation metric 'epoch_loss'=5.313974
INFO:root:Epoch[184] Learning rate is 5e-05
100%|██████████| 200/200 [00:18<00:00, 11.09it/s, avg_epoch_loss=5.38]
INFO:root:Epoch[184] Elapsed 

INFO:root:Epoch[216] Elapsed time 17.649 seconds
INFO:root:Epoch[216] Evaluation metric 'epoch_loss'=5.302865
INFO:root:Epoch[217] Learning rate is 5e-05
100%|██████████| 200/200 [00:17<00:00, 11.56it/s, avg_epoch_loss=5.34]
INFO:root:Epoch[217] Elapsed time 17.314 seconds
INFO:root:Epoch[217] Evaluation metric 'epoch_loss'=5.335404
INFO:root:Epoch[218] Learning rate is 5e-05
100%|██████████| 200/200 [00:18<00:00, 11.03it/s, avg_epoch_loss=5.34]
INFO:root:Epoch[218] Elapsed time 18.137 seconds
INFO:root:Epoch[218] Evaluation metric 'epoch_loss'=5.341279
INFO:root:Epoch[219] Learning rate is 5e-05
100%|██████████| 200/200 [00:18<00:00, 10.70it/s, avg_epoch_loss=5.34]
INFO:root:Epoch[219] Elapsed time 18.703 seconds
INFO:root:Epoch[219] Evaluation metric 'epoch_loss'=5.344119
INFO:root:Epoch[220] Learning rate is 5e-05
100%|██████████| 200/200 [00:18<00:00, 10.80it/s, avg_epoch_loss=5.34]
INFO:root:Epoch[220] Elapsed time 18.518 seconds
INFO:root:Epoch[220] Evaluation metric 'epoch_loss'

{'MASE': 3.63534742,
 'MSIS': 42.68165088,
 'epochs': 250,
 'sMAPE': 0.03389154,
 'wQuantileLoss[0.5]': 0.03102737,
 'wQuantileLoss[0.9]': 0.01364864}


In [5]:
res = main(data="m4_daily", seed=42, epochs=300, batches=200)
pprint(res)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 97983
INFO:root:Epoch[0] Learning rate is 0.001
 50%|████▉     | 99/200 [00:14<00:14,  6.97it/s, avg_epoch_loss=7.14]


KeyboardInterrupt: 

In [ ]:
res = main(data="m4_daily", seed=42, epochs=350, batches=200)
pprint(res)

In [ ]:
res = main(data="m4_daily", seed=42, epochs=400, batches=200)
pprint(res)

In [ ]:
res = main(data="m4_daily", seed=43, epochs=100, batches=200)
pprint(res)

In [ ]:
res = main(data="m4_daily", seed=43, epochs=150, batches=200)
pprint(res)

In [ ]:
res = main(data="m4_daily", seed=43, epochs=200, batches=200)
pprint(res)

In [ ]:
res = main(data="m4_daily", seed=43, epochs=250, batches=200)
pprint(res)

In [ ]:
res = main(data="m4_daily", seed=43, epochs=300, batches=200)
pprint(res)

In [ ]:
res = main(data="m4_daily", seed=43, epochs=350, batches=200)
pprint(res)

In [ ]:
res = main(data="m4_daily", seed=43, epochs=400, batches=200)
pprint(res)

In [5]:
# results = []

# for i in range(100, 401, 50):
#     print("Epochs:", i)
#     res = main(data="m4_daily", seed=44, epochs=i, batches=100)
#     pprint(res)
#     results.append(res)
    
# results

In [ ]:
# results = []

# for i in range(100, 401, 50):
#     print("Epochs:", i)
#     res = main(data="m4_daily", seed=42, epochs=i, batches=200)
#     pprint(res)
#     results.append(res)
    
# results

In [ ]:
# results = []

# for i in range(100, 401, 50):
#     print("Epochs:", i)
#     res = main(data="m4_daily", seed=43, epochs=i, batches=200)
#     pprint(res)
#     results.append(res)
    
# results

In [ ]:
# results = []

# for i in range(100, 401, 50):
#     print("Epochs:", i)
#     res = main(data="m4_daily", seed=44, epochs=i, batches=200)
#     pprint(res)
#     results.append(res)
    
# results